- 데이터 시각화 및 분석
  - 이상치
    - 통계적관점
      - 데이터가 평균에서 벗어난 정도를 나타내는 표준편차를 기준으로 정의 예들을어서 평균에 3표준 편차 이상 벗어난 값
      - IQR(Interquartile Range) 사용해서 Q1 - 1.5xIQR and Q3 + 1.5xIQR
    - 도메인 기반 정의
      - 도메인 지식에 따라서 특정 임계값을 초과하거나 미만인 값을 이상치로 간주

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 임의의 데이터 생성
np.random.seed(42)
data = np.random.normal(50,10,100)  # 평균 50, 표준편차 10인 정규분포 데이터
print(data)
data = np.append(data, [10,100])  # 이상치를 추가
print(data)

[54.96714153 48.61735699 56.47688538 65.23029856 47.65846625 47.65863043
 65.79212816 57.67434729 45.30525614 55.42560044 45.36582307 45.34270246
 52.41962272 30.86719755 32.75082167 44.37712471 39.8716888  53.14247333
 40.91975924 35.87696299 64.65648769 47.742237   50.67528205 35.75251814
 44.55617275 51.1092259  38.49006423 53.75698018 43.9936131  47.0830625
 43.98293388 68.52278185 49.86502775 39.42289071 58.22544912 37.7915635
 52.08863595 30.40329876 36.71813951 51.96861236 57.3846658  51.71368281
 48.84351718 46.98896304 35.2147801  42.80155792 45.39361229 60.57122226
 53.4361829  32.36959845 53.24083969 46.1491772  43.23078    56.11676289
 60.30999522 59.31280119 41.60782477 46.90787624 53.31263431 59.75545127
 45.20825762 48.14341023 38.93665026 38.03793376 58.12525822 63.56240029
 49.27989878 60.03532898 53.61636025 43.54880245 53.61395606 65.38036566
 49.64173961 65.64643656 23.80254896 58.21902504 50.87047068 47.0099265
 50.91760777 30.12431085 47.80328112 53.57112572 64.77

In [ ]:
# 데이터 프레임 생성
df = pd.DataFrame(data, columns=['value'])
df.head()

,value
0,54.967142
1,48.617357
2,56.476885
3,65.230299
4,47.658466


In [ ]:
# IQR 기반 이상치 탐지
Q1 = df.value.quantile(0.25)
Q3 = df.value.quantile(0.75)
IQR = Q3 - Q1
print(Q1, Q3, IQR)

43.98560368348251 54.664601193448426 10.678997509965917


In [5]:
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(lower_bound, upper_bound)

27.967107418533633 70.6830974583973


In [10]:
# 이상치 표시
df['is_outlier'] = (df.value < lower_bound) | (df.value > upper_bound)

# 결과출력
print('IQR:', IQR)
print("lower bound : ", lower_bound)
print("upper bound : ", upper_bound)
print("Detected outliers:\n", df[df['is_outlier']])

IQR: 10.678997509965917
lower bound :  27.967107418533633
upper bound :  70.6830974583973
Detected outliers:
           value  is_outlier
74    23.802549        True
100   10.000000        True
101  100.000000        True


- 실제 사례
  - 금융
    - 신용카드 사기 탐지에서 비정상적으로 높은 거래 금액
  - 의료
    - 환자의 생체 데이터를 분석할 때 비정상적으로 높은 혈압이나 체온
  - 제조
    - 센서 데이터에서 비정상적인 값으로 기계 결함을 감지

  - 상관관계
  - matplotlib, seaboarn